In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from operator import itemgetter
from itertools import cycle, islice
import pandas as pd
import numpy as np
import sif_embedding_wrapper
import utils
import itertools
import os

In [2]:
# from gensim.models.keyedvectors import KeyedVectors

# model = KeyedVectors.load_word2vec_format('/home/stirunag/pre-trained_word_embeddings/PubMed-and-PMC-w2v.bin', binary=True)
# model.save_word2vec_format('/home/stirunag/pre-trained_word_embeddings/PubMed-and-PMC-w2v.txt', binary=False)

# words, embs, weight4ind = sif_embedding_wrapper.load_embeddings("/home/stirunag/pre-trained_word_embeddings/PubMed-and-PMC-FS.txt", 
#                                                      '/home/stirunag/pre-trained_word_embeddings/wiki/enwiki_vocab_min200.txt')

words, embs, weight4ind = sif_embedding_wrapper.load_embeddings("/home/stirunag/pre-trained_word_embeddings/glove/glove.6B.50d.txt", 
                                                      '/home/stirunag/pre-trained_word_embeddings/wiki/enwiki_vocab_min200.txt')

In [3]:
# get the current working directory
data_path = os.path.abspath(os.path.join(os.path.dirname( '__file__' ), '..', 'Datasets'))+'/'

# Although the dataset says csv, it is tab delimited. In addition to this, they have severe codels problems. 
# So best to parse throught codes first. 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfc in position 2: invalid start byte

#open for reading with "universal" type set

import codecs

doc_d_t = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_drug_target'+'.csv','rU','UTF-8') 
EUADR_drug_target = pd.read_csv(doc_d_t, sep='\t', na_filter = False)
EUADR_drug_target['CLASS'] = 'drug_gene'

doc_t_d = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_target_disease'+'.csv','rU','UTF-8',errors='ignore') 
EUADR_target_disease = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
EUADR_target_disease['CLASS'] = 'gene_disease'
       
doc_d_d = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_drug_disease'+'.csv','rU','UTF-8')                       
EUADR_drug_disease = pd.read_csv(doc_d_d, sep='\t', na_filter = False)
EUADR_drug_disease['CLASS'] = 'drug_disease'



In [4]:
df = EUADR_drug_target.append(EUADR_target_disease).append(EUADR_drug_disease)
df.reset_index(inplace=True)


In [5]:
doc_embeddings = sif_embedding_wrapper.sentences2vecs(df["SENTENCE"], embs, words, weight4ind)
df["vector"] = pd.Series(list(doc_embeddings))


In [23]:
ground_truth = {}

for idx, row in df.iterrows():
    doc_id = idx
    ground_truth[doc_id] = row['ASSOCIATION_TYPE']

ground_truth

inv_map = {}
for k, v in ground_truth.items():
    inv_map[v] = inv_map.get(v, [])
    inv_map[v].append(k)
    
inv_map  

{'PA': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  22,
  24,
  26,
  27,
  29,
  30,
  33,
  35,
  37,
  39,
  40,
  42,
  43,
  44,
  48,
  49,
  50,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  61,
  62,
  64,
  66,
  67,
  68,
  69,
  70,
  71,
  73,
  76,
  77,
  78,
  80,
  82,
  83,
  84,
  98,
  100,
  102,
  105,
  106,
  107,
  109,
  110,
  111,
  118,
  119,
  120,
  122,
  123,
  124,
  125,
  126,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  147,
  148,
  151,
  153,
  155,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166,
  167,
  168,
  170,
  171,
  172,
  173,
  174,
  176,
  183,
  185,
  186,
  187,
  188,
  190,
  192,
  193,
  195,
  196,
  197,
  199,
  200,
  202,
  203,
  204,
  207,
  208,
  209,
  210,
  213,
  216,
  217,
  218,
  219,
  220,
  221,
  222,
  224,
  229,
  231,
  232,
  233,
  237,
  238,
  239,
  

In [7]:
categories = list(df["ASSOCIATION_TYPE"].unique())
categories

['PA', 'SA', 'FA', 'NA']

In [36]:
categories_ = []
# category_reps = {categories[0]: ['188'], categories[1]: ["585"]}
# category_reps = {'PA': ['1','2','3', '17'], 'SA': ['11','28','34', '72'], 
#                  'FA': ['20', '21', '23', '31'], 'NA': ['150', '179', '201', '230']}
                 
category_reps = inv_map

for k,v in category_reps.items():
    categories_.append(k)

In [38]:
category_reps[c]

[150,
 179,
 201,
 205,
 206,
 230,
 352,
 353,
 386,
 387,
 389,
 422,
 425,
 428,
 429,
 433,
 435,
 436,
 437,
 438,
 502,
 505,
 508,
 572,
 575,
 728,
 824]

In [41]:
category_vecs = {}
for c in categories_:
    vectors = np.asarray(list(df.loc[df.index.isin(category_reps[c])].vector))
    category_vecs[c] = np.mean(vectors, axis=0)

category_vecs    

{'PA': array([ 0.41236051, -0.05944873,  0.13004648, -0.09951285, -0.12618287,
         0.58114858,  0.42657313, -0.14856458,  0.33515388,  0.42100655,
         0.45350058, -0.08313132,  0.12578655, -0.09841819, -0.03464867,
         0.13360661, -0.27252536, -0.18523202,  0.03744354,  0.08392766,
        -0.25986083, -0.24476628,  0.25568928, -0.0194959 , -0.05098169,
        -0.16427464, -0.00176145, -0.13133206,  0.03779821,  0.15203096,
         0.88273723, -0.02017684,  0.19159335, -0.53791566,  0.05021709,
        -0.04242201,  0.28711816,  0.11221867, -0.054385  ,  0.05320518,
         0.05421387, -0.12679378, -0.05664655,  0.20739146,  0.17822269,
        -0.14674712,  0.21655151,  0.08015338, -0.11534325,  0.23621486]),
 'SA': array([ 0.53658508, -0.04563028,  0.06404406, -0.10213333, -0.12534395,
         0.59067587,  0.34990257, -0.16393354,  0.35053804,  0.39726501,
         0.41776906, -0.02898027,  0.13261967, -0.14064506, -0.01802243,
         0.19656583, -0.20451296, -0.

In [42]:
predictions = {}

for idx, row in df.iterrows():
    doc_id = idx
    max_sim = 0
    winner = categories[0]
    for j in category_vecs:
        sim = cosine_similarity(row["vector"].reshape(1, -1), category_vecs[j].reshape(1, -1)).flatten()[0]
        if sim > max_sim:
            max_sim = sim
            winner = j
    predictions[doc_id] = winner
    

In [43]:
predictions

{0: 'PA',
 1: 'FA',
 2: 'PA',
 3: 'PA',
 4: 'FA',
 5: 'PA',
 6: 'FA',
 7: 'FA',
 8: 'FA',
 9: 'PA',
 10: 'PA',
 11: 'PA',
 12: 'PA',
 13: 'PA',
 14: 'PA',
 15: 'PA',
 16: 'FA',
 17: 'FA',
 18: 'PA',
 19: 'SA',
 20: 'FA',
 21: 'FA',
 22: 'PA',
 23: 'FA',
 24: 'FA',
 25: 'FA',
 26: 'NA',
 27: 'PA',
 28: 'NA',
 29: 'FA',
 30: 'FA',
 31: 'FA',
 32: 'PA',
 33: 'PA',
 34: 'SA',
 35: 'NA',
 36: 'NA',
 37: 'NA',
 38: 'PA',
 39: 'PA',
 40: 'SA',
 41: 'FA',
 42: 'FA',
 43: 'FA',
 44: 'PA',
 45: 'FA',
 46: 'FA',
 47: 'FA',
 48: 'FA',
 49: 'FA',
 50: 'NA',
 51: 'FA',
 52: 'NA',
 53: 'FA',
 54: 'PA',
 55: 'PA',
 56: 'SA',
 57: 'PA',
 58: 'PA',
 59: 'PA',
 60: 'PA',
 61: 'PA',
 62: 'NA',
 63: 'FA',
 64: 'FA',
 65: 'SA',
 66: 'SA',
 67: 'PA',
 68: 'FA',
 69: 'FA',
 70: 'FA',
 71: 'FA',
 72: 'FA',
 73: 'FA',
 74: 'FA',
 75: 'FA',
 76: 'FA',
 77: 'FA',
 78: 'FA',
 79: 'FA',
 80: 'FA',
 81: 'PA',
 82: 'FA',
 83: 'FA',
 84: 'FA',
 85: 'FA',
 86: 'FA',
 87: 'FA',
 88: 'FA',
 89: 'FA',
 90: 'FA',
 91: 'FA'

In [44]:
from sklearn.metrics import f1_score

def get_accuracy_score(predictions, truth_dict):
    scores = []
    preds = []
    labels = []
    
    for k,v in predictions.items():
    
        preds.append(v)
        labels.append(truth_dict[k])

#         if v == truth_dict[k]:
#             scores.append(1)
#         else:
#             scores.append(0)
    
#     print(f1_score(labels, preds, average='weighted')) 
#     if len(scores) == 0:
#       return 0.0
#     return sum(scores) / float(len(scores))

    return f1_score(labels, preds, average='weighted')

get_accuracy_score(predictions, ground_truth)




0.36019241718597433

In [ ]:
from itertools import cycle
from collections import deque
    
    
def infer_topics(docs, n_topics, min_text_length=80, max_iter=150, batch_size=128, learning_offset=300.):
    unclassifiable = list(docs[docs["text"].map(len) < min_text_length].doc_id)
    filtered = docs[~docs['doc_id'].isin(unclassifiable)]
    ids = [d for d in list(filtered.doc_id)[0:10]]
    n_features = 1000
    tf_vectorizer = TfidfVectorizer(
        stop_words='english',
        max_df=0.95,
        min_df=0.1,
        max_features=n_features)
    tf = tf_vectorizer.fit_transform(list(filtered.loc[:, 'text']))
    lda = LatentDirichletAllocation(
        n_components=n_topics,
        max_iter=max_iter,
        batch_size=batch_size,
        learning_method='online',
        learning_offset=learning_offset,
        random_state=0)
    lda.fit(tf)
    doc_topics = lda.transform(tf)
    topic_leaders = {"topic_{}".format(i): [] for i in iter(range(n_topics))}
    for idx, probs in enumerate(doc_topics):
        score = max(probs)
        topic = np.argmax(probs)

        doc_id = filtered.loc[filtered.index[idx]].doc_id
        topic_leaders["topic_{}".format(topic)].append(
            {"doc_id": doc_id, "score": score})
    for i in iter(range(n_topics)):
        topic_leaders["topic_{}".format(i)] = sorted(
            topic_leaders["topic_{}".format(i)], key=itemgetter('score'), reverse=True)

    def roundrobin(*iterables):
        q = deque(iter(it) for it in iterables)
        for itr in cycle(q):
            try:
                yield itr.__next__()
            except StopIteration:
                if len(q) > 0:
                    q.pop()
                else:
                    break

    return list(roundrobin(*topic_leaders.values()))

ordered_docs = infer_topics(df, 3)

In [ ]:
# {'drug_gene': ['1', '2', '3', '124'],
#  'gene_disease': ['259', '260', '273', '275'],
#  'drug_disease': ['827', '837', '817', '820', '840', '845']}

ordered_docs

In [ ]:
df[df['doc_id']=='261']

In [ ]:
def auto_classify(docs, category_reps, min_text_length=80):
    # Exclude docs deemed too short to classify.
    skip_prediction = list(df[df["text"].map(len) < min_text_length].doc_id)
    categories = []
    for k,v in category_reps.items():
        categories.append(k)
        skip_prediction.extend(v) # No need to predict manually labeled docs
    category_vecs = {}
    for c in categories:
        vectors = np.asarray(list(docs.loc[docs['doc_id'].isin(category_reps[c])].vector))
        category_vecs[c] = np.mean(vectors, axis=0)

    predictions = {}
    for idx, row in docs.iterrows():
        doc_id = row["doc_id"]
        if doc_id in skip_prediction:
            continue
        max_sim = 0
        winner = categories[0]
        for j in category_vecs:
            sim = cosine_similarity(row["vector"].reshape(1, -1), category_vecs[j].reshape(1, -1)).flatten()[0]
            if sim > max_sim:
                max_sim = sim
                winner = j
        predictions[doc_id] = winner
    return predictions

In [ ]:
representatives = {c:[] for c in categories_}
top_n = 25
ordered_ids = [d["doc_id"] for d in ordered_docs]
for doc_id in ordered_ids[:top_n]:
    gt = truth_dict[str(doc_id)]
    representatives[gt].append(doc_id)

    
  

values = [representatives[c] for c in categories]

doc_combs = list(itertools.product(*values))    

doc_combs

In [ ]:
representatives

In [ ]:
accuracies =[]
for comb in doc_combs:
    category_reps = {}
    for i,c in enumerate(categories):
        category_reps[c] = [str(comb[i])]

    preds = auto_classify(docs, category_reps)
    acc = get_accuracy_score(preds, truth_dict)
    accuracies.append(acc)

In [ ]:
for comb in doc_combs:
    category_reps = {}
    for i,c in enumerate(categories):
        category_reps[c] = [str(comb[i])]

category_reps        

In [ ]:
max(accuracies)

In [ ]:
def get_top_lda_combs(ordered_ids, docs_df, categories, truth_dict, top_n=12):
    representatives = {c:[] for c in categories}
    for doc_id in ordered_ids[:top_n]:
        gt = truth_dict[str(doc_id)]
        representatives[gt].append(doc_id)
    for c in categories:
        if len(representatives[c]) == 0:
            print("No representatives for %s" % c)
            return None
    values = [representatives[c] for c in categories]
    doc_combs = list(itertools.product(*values))
    return doc_combs

def get_lda_accuracies(categories, doc_combs, docs_df, truth_dict):
    accuracies = []
    for comb in doc_combs:
        category_reps = {}
        for i,c in enumerate(categories):
            category_reps[c] = [str(comb[i])]
        preds = auto_classify(docs_df, category_reps)
        acc = get_accuracy_score(preds, truth_dict)
        accuracies.append(acc)
    return accuracies


top_lda_combs = get_top_lda_combs([d["doc_id"] for d in ordered_docs], 
                                  df, categories, truth_dict)
lda_accs = get_lda_accuracies(categories, top_lda_combs, df, truth_dict)
max(lda_accs)